In [ ]:
functions-framework==3.*
SQLAlchemy==1.4.37
PyMySQL==1.0.2
pandas==1.5.2
requests==2.26.0
beautifulsoup4==4.10.0
pytz==2021.3
google-cloud-secret-manager==2.8.0

In [ ]:
import functions_framework
import pandas as pd
import sqlalchemy
import os
from google.cloud import secretmanager_v1beta1 as secretmanager
import json
import time



@functions_framework.http
def insert(request):
    api_key, sql_password, api_key_airport = get_secrets()
    conn = connection(api_key, sql_password, api_key_airport)
    weather_request(api_key, sql_password, api_key_airport)
    airport_schedule(api_key, sql_password, api_key_airport)
    return 'Data successfully added'

def get_secrets():
    # Obtén la información del proyecto desde la configuración del entorno
    project_id = os.environ.get('My_First_Project')
    
    # Nombres de los secretos en Secret Manager
    api_key_secret_name = 'API_key'
    sql_password_secret_name = 'Password_SQL_instance'
    api_key_airport_secret_name = 'API_key_airport'

    # Configura el cliente de Secret Manager
    client = secretmanager.SecretManagerServiceClient()

    # Obtén el secreto de la API Key desde Secret Manager
    api_key_secret_path = f"projects/{project_id}/secrets/{api_key_secret_name}/versions/latest"
    api_key_response = client.access_secret_version(name=api_key_secret_path)
    api_key = api_key_response.payload.data.decode("UTF-8")

    # Obtén el secreto de la contraseña SQL desde Secret Manager
    sql_password_secret_path = f"projects/{project_id}/secrets/{sql_password_secret_name}/versions/latest"
    sql_password_response = client.access_secret_version(name=sql_password_secret_path)
    sql_password = sql_password_response.payload.data.decode("UTF-8")

    # Obtén el secreto de la API Key Airport desde Secret Manager
    api_key_airport_secret_path = f"projects/{project_id}/secrets/{api_key_airport_secret_name}/versions/latest"
    api_key_airport_response = client.access_secret_version(name=api_key_airport_secret_path)
    api_key_airport = api_key_airport_response.payload.data.decode("UTF-8")

    return api_key, sql_password, api_key_airport

def connection(api_key, sql_password, api_key_airport):
    connection_name = "warm-melody-410609:europe-west1:wbs-mysql-db"
    db_user = "root"
    db_password = sql_password
    schema_name = "gans"

    driver_name = 'mysql+pymysql'
    query_string = {"unix_socket": f"/cloudsql/{connection_name}"}

    db = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL(
            drivername=driver_name,
            username=db_user,
            password=db_password,
            database=schema_name,
            query=query_string,
        )
    )
    return db

def weather_request(api_key, sql_password, api_key_airport):
    import pandas as pd
    from datetime import datetime, timedelta
    import requests
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    import re
    import pandas as pd
    from datetime import datetime
    from pytz import timezone
    
    information_df = pd.read_sql("information", con = connection(api_key, sql_password, api_key_airport))
    cities_df = pd.read_sql("cities",con= connection(api_key, sql_password, api_key_airport))
    information_df = information_df.merge(cities_df,
                                          on = "city_id",
                                          how = "left"
                                         )
    
    berlin_timezone = timezone('Europe/Berlin')
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    weather_items = []
                            
    for city in information_df["city_name"]:
        latitude = information_df.loc[information_df["city_name"] == city, "latitude"].values[0]
        longitude = information_df.loc[information_df["city_name"] == city, "longitude"].values[0]

        # Construct the API request URL
        url_map = f"http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}"
    
        # Make the API request and handle potential errors
        try:
            city_id_openweather = requests.get(url_map)
            city_id_openweather.raise_for_status()  # Raise an error for bad responses (non-2xx status codes)
            city_id_openweather = city_id_openweather.json()
            city_id_openweather = city_id_openweather["id"]
        
        except requests.exceptions.RequestException as e:
            # Handle request exceptions (e.g., network issues, API errors)
            print(f"Error in request for latitude {latitude}, longitude {longitude}: {e}")
            temperature_request = None  # Append None for temperature to indicate an error
            sky_request = None  # Append None for sky description to indicate an error
    
        url = f"http://api.openweathermap.org/data/2.5/forecast?id={city_id_openweather}&appid={api_key}"
 
        response = requests.get(url)
    
        response = response.json()
    
        for item in response["list"]:
            weather_item = {
                "city_name" : city,
                "forecast_time": item.get("dt_txt", None),
                "temperature": item["main"].get("temp", None),
                "forecast": item["weather"][0].get("main", None),
                "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                "wind_speed": item["wind"].get("speed", None),
                "retrieval_time" : retrieval_time
            }
        
            weather_items.append(weather_item)
            
    weather_df = pd.DataFrame(weather_items)
    
    weather_df = weather_df.merge(cities_df,
                                  on = "city_name",
                                  how = "left")
    
    weather_df = weather_df.drop("city_name",
                                 axis = 1
                                )
    weather_df.to_sql('weather',
                      if_exists='append',
                      con=connection(api_key, sql_password, api_key_airport),
                      index=False
                     )

def airport_schedule(api_key, sql_password, api_key_airport):
    from datetime import datetime, timedelta
    import requests
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    import re
    import pandas as pd
    from datetime import datetime
    from pytz import timezone
    
    airport_arrivals = []
    iata_names = []  # Corrected variable name
    airlines = []
    status = []
    
    airports_final = pd.read_sql("airports", con=connection(api_key, sql_password, api_key_airport))
    
    today = datetime.now().date().strftime("%Y-%m-%dT")
    tomorrow = (datetime.now().date() + timedelta(days=1)).strftime("%Y-%m-%dT")
    berlin_timezone = timezone('Europe/Berlin')
    
    current_time_berlin = datetime.now(berlin_timezone)
    time_in_12_hours = current_time_berlin + timedelta(hours=12)
    current_time_berlin_formated = current_time_berlin.strftime("%H:%M")
    time_in_12_hours_formated = time_in_12_hours.strftime("%H:%M")
        
    for airport in airports_final["Airport_IATA"]:
        
        if current_time_berlin.hour < 12:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{airport}/{today}{current_time_berlin_formated}/{today}{time_in_12_hours_formated}"
        else:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{airport}/{today}{current_time_berlin_formated}/{tomorrow}{time_in_12_hours_formated}"

        querystring = {"withLeg": "true", "withCancelled": "true", "withCodeshared": "true", "withCargo": "true",
               "withPrivate": "true", "withLocation": "false"}

        headers = {
            "X-RapidAPI-Key": f"{api_key_airport}",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            arrivals = response.json()["arrivals"]

            for arrival in arrivals:
                if "arrival" in arrival and "scheduledTime" in arrival["arrival"] and "local" in arrival["arrival"]["scheduledTime"]:
                    airport_arrivals.append(arrival["arrival"]["scheduledTime"]["local"])
                    iata_names.append(airport)  # Using the correct variable name airport
           
                    if "airline" in arrival and "name" in arrival["airline"]:
                        airlines.append(arrival["airline"]["name"])
                    else:
                        airlines.append(None)
                
                    if "status" in arrival:
                        status.append(arrival["status"])
                    else:
                        status.append(None)
        else:
            # Append None values for all arrays if the response status is not 200
            airport_arrivals.append(None)
            iata_names.append(None)
            airlines.append(None)
            status.append(None)
        
    airport_schedule_df = pd.DataFrame({"airport_arrivals": airport_arrivals, "airlines": airlines, "iata_name": iata_names, "status": status})
    
    airport_schedule_df = airport_schedule_df.merge(airports_final,
                              left_on = "iata_name",
                              right_on = "Airport_IATA",
                              how = "left"
                             )
    
    airport_schedule_df = airport_schedule_df.drop(["iata_name", "Airport_IATA"], axis=1).rename(columns={"status": "status_"})
    
    airport_schedule_df['airport_arrivals'] = pd.to_datetime(airport_schedule_df['airport_arrivals'], utc=True)

    # Format the datetime column as a string in the desired format
    airport_schedule_df['airport_arrivals'] = airport_schedule_df['airport_arrivals'].dt.strftime(
        '%Y-%m-%d %H:%M:%S')

    airport_schedule_df.to_sql('airport_sheduldes',
                                  if_exists='append',
                                  con=connection(api_key, sql_password, api_key_airport),
                                  index=False,
                                  )
    